# Filtering Piano Instrument - Sound Wave & Feedfoward Neural Network (FNN)
![Getting Started](./PianoSoundWaveFNN/images/SoundWaveFNN.jpg)

****
## Goal
* What types of sounds are generated?
* What are the features used to train the system?
* What is the DL architecture employeed?
* What are the inputs for generation?

****

In [ ]:
import os
import librosa
import librosa.display
import IPython.display as ipd
import numpy as np
import matplotlib.pyplot as plt
import soundfile as sf
import keras
from keras.layers import Dense, Dropout, PReLU
from keras.optimizers import Adam
import pickle

DATASET_PATH = './PianoSoundWaveFNN/data/audio/train/piano/10'
OUTPUT_PATH = os.path.join(DATASET_PATH, 'pkl')
MODEL_PATH = './PianoSoundWaveFNN/model'

WAVE_SAMPLE_RATE = 44100
NUM_SAMPLES = 1024

## Functions to extract training dataset
****

In [ ]:
def get_WaveSamples_from_Wave(wave_file_path, output_data_dir, num_samples=NUM_SAMPLES, sr=WAVE_SAMPLE_RATE):
    filename = os.path.basename(wave_file_path)
    f_token = filename.split('.')[0]
    data_file_name = f_token + '.pkl'

    # create segment directory
    if not os.path.exists(output_data_dir):
        os.makedirs(output_data_dir)
    
    dataset_path = os.path.join(output_data_dir, data_file_name)
    if os.path.exists(dataset_path):
        # retrieve the data and return
        file = open(dataset_path, 'rb')
        print("----- getting dataset from {}".format(dataset_path))
        w_signal = pickle.load(file)
        file.close()
        return w_signal

    dataset = []
    signal, sr = librosa.load(wave_file_path, sr=sr)
    num_segments = int(len(signal)/num_samples)
            
    # process all segments of audio file
    for d in range(num_segments):
        # calculate start and finish sample for current segment
        start = num_samples * d
        finish = start + num_samples
        dataset.append(signal[start:finish])

    w_signal = np.array(dataset).reshape(-1, num_samples)

    print(w_signal.shape)

    # save to pickle file
    file = open(dataset_path, 'wb')
    print("----- generating dataset {}".format(dataset_path))
    pickle.dump(w_signal, file)
    file.close()
    return w_signal
    
def create_WSamples(dataset_path, data_output_dir, num_samples=NUM_SAMPLES, sr=WAVE_SAMPLE_RATE):    
    if not os.path.exists(data_output_dir):
        os.makedirs(data_output_dir)

    for dirpath, dirnames, filenames in os.walk(dataset_path):
        if dirpath == data_output_dir:
            print('----- skipping {}'.format(dirpath))
            continue

        print('----- processing {}'.format(dirpath))
        # process all audio files in genre sub-dir
        for f in filenames:
            # load all audio file
            if "mix" in f:
                m_wave_file_path = os.path.join(dataset_path, f)
                m_signal = get_WaveSamples_from_Wave(m_wave_file_path, data_output_dir, num_samples=num_samples, sr = sr)

                t_token = f.split('_')[2]
                t_file = t_token + '.wav'
                t_wave_file_path = os.path.join(dirpath,  t_file)
                t_signal = get_WaveSamples_from_Wave(t_wave_file_path, data_output_dir, num_samples=num_samples, sr = sr)
    return

def create_traing_dataset(dataset_path, dataset_files):   
    F_input_file_path = os.path.join(dataset_path, 'input.pkl')
    F_target_file_path = os.path.join(dataset_path,'target.pkl')

    input_Dataset = np.array([])
    target_Dataset = np.array([])
    input_dataset = []
    target_dataset = []
        
    if os.path.exists(F_input_file_path) and os.path.exists(F_input_file_path) :
        print("----- getting input dataset from {}".format(F_input_file_path))
        file = open(F_input_file_path, 'rb')
        input_Dataset = pickle.load(file)
        file.close()

        print("----- getting target dataset from {}".format(F_target_file_path))
        file = open(F_target_file_path, 'rb')
        target_Dataset = pickle.load(file)
        file.close()
        return input_Dataset, target_Dataset
    

    for f in dataset_files:
        input_file_path = os.path.join(dataset_path, f)
        print("----- getting input dataset from {}".format(input_file_path))
        file = open(input_file_path, 'rb')
        Dataset = pickle.load(file)
        file.close()
        input_dataset.append(Dataset.tolist())

        t_token = f.split('_')[2]
        tf = t_token + '.pkl'
        target_file_path = os.path.join(dataset_path, tf)
        print("----- getting target dataset from {}".format(target_file_path))
        file = open(target_file_path, 'rb')
        Dataset = pickle.load(file)
        file.close()
        target_dataset.append(Dataset.tolist())

    num_samples = Dataset.shape[1]

    input_Dataset = np.array(input_dataset).reshape(-1, num_samples)
    target_Dataset = np.array(target_dataset).reshape(-1, num_samples)

    #save to pickle file
    file = open(F_input_file_path, 'wb')
    print("----- generating input dataset {}".format(F_input_file_path))
    pickle.dump(input_Dataset, file)
    file.close()

    file = open(F_target_file_path, 'wb')
    print("----- generating target dataset {}".format(F_target_file_path))
    pickle.dump(target_Dataset, file)
    file.close()

    return input_Dataset, target_Dataset

## Train Model with Dataset
****

In [ ]:
if not os.path.exists(MODEL_PATH):
    create_WSamples(DATASET_PATH, OUTPUT_PATH, num_samples=NUM_SAMPLES, sr=WAVE_SAMPLE_RATE)  
    input_list = [
        'mix_10_piano1_10_violin1.pkl', \
        'mix_10_piano1_10_violin2.pkl', \
        'mix_10_piano1_10_violin3.pkl', \
        'mix_10_piano2_10_violin1.pkl', \
        'mix_10_piano2_10_violin2.pkl', \
        'mix_10_piano2_10_violin3.pkl']

    input, target = create_traing_dataset(OUTPUT_PATH, input_list)

    print('---------- {}'.format(input.shape))

    # create model
    input_shape = input[0].shape
    input_size = input[0].shape[0]
    output_size = target[0].shape[0]
    
    model = keras.models.Sequential()
    model.add(Dense(input_size, input_shape=input_shape))
    model.add(PReLU())
    model.add(Dense(512))
    model.add(PReLU())
    model.add(Dense(output_size))
    model.compile(Adam(), 'mse')
    model.summary()
    model.fit(input, target, epochs=10)
    model.save(MODEL_PATH)
else:
    model = keras.models.load_model(MODEL_PATH)

In [ ]:
def x_fade_profile(batch_dim):
    x = np.arange(batch_dim)
    return 1 - abs(x - (batch_dim / 2)) / (batch_dim / 2)

    
def model_predict(model, input_track, num_samples):
    dim = num_samples
    n_batches = int(len(input_track) / dim) - 1
    pred_batches = input_track[0:n_batches*dim].reshape((-1, dim))
    pred_batches_shifted = input_track[dim//2:n_batches*dim + dim//2].reshape((-1, dim))
    
    xfp = x_fade_profile(dim)
    x0 = np.array([xfp * batch for batch in model.predict(pred_batches)]).reshape(-1)
    x1 = np.array([xfp * batch for batch in model.predict(pred_batches_shifted)]).reshape(-1)
    return x0 + x1

## Test Model
****

### Prepare Test Data

In [ ]:
input_test_file = os.path.join(DATASET_PATH, 'mix_10_piano2_10_violin2.wav')
target_test_file = os.path.join(DATASET_PATH, 'piano2.wav')

input_test_signal, sr = librosa.load(input_test_file, sr = WAVE_SAMPLE_RATE)
target_test_signal, sr = librosa.load(target_test_file, sr = WAVE_SAMPLE_RATE)

### Input Track

In [ ]:
ipd.Audio(input_test_signal, rate=WAVE_SAMPLE_RATE)

### Target Track

In [ ]:
ipd.Audio(target_test_signal, rate=WAVE_SAMPLE_RATE)

### Prediciton Track

In [ ]:
ipd.Audio(model_predict(model, input_test_signal, num_samples=NUM_SAMPLES), rate=WAVE_SAMPLE_RATE)